In [1]:
!pip install langchain langchain_core langchain_community yandex_chain langchain_chroma langchain_huggingface

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFacePipeline
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

os.environ['HF_TOKEN'] = ...

# Загрузка и подготовка данных (выполняется один раз для заполнения базы)
def init_chroma_db(documents, persist_directory="./chroma_db"):
    embeddings = HuggingFaceEndpointEmbeddings(model="mixedbread-ai/mxbai-embed-large-v1", task="feature-extraction")

    vectordb = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    return vectordb

# Инициализация модели Qwen для генерации текста
def load_qwen_model():
    model_name = "Qwen/Qwen2.5-0.5B"

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto",
        trust_remote_code=True
    )

    # Создаем пайплайн для генерации
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.3,
        top_p=0.9,
        repetition_penalty=1.1,
        return_full_text=False
    )

    llm = HuggingFacePipeline(pipeline=pipe)
    return llm

# Обработка входящего письма
def process_email(input_email, vectordb, k=3):

    similar_docs = vectordb.similarity_search(input_email, k=k)

    # Формирование контекста из найденных писем и ответов
    context = "\n\n".join([
        f"Письмо: {doc.page_content}\nОтвет: {doc.metadata.get('response', '')}"
        for doc in similar_docs
    ])

    return context

# Генерация ответа через модель Qwen
def generate_response(input_email, context, llm):
    prompt_template = PromptTemplate(
        input_variables=["email", "context"],
        template="""<|im_start|>system
Вы специалист службы поддержки. На основе исторической переписки ответьте на новое письмо в похожем стиле.

Исторические примеры:
{context}<|im_end|>
<|im_start|>user
Новое письмо для ответа:
{email}

Пожалуйста, составьте ответ в стиле приведенных примеров, но :<|im_end|>
<|im_start|>assistant
"""
    )

    chain = (
        {"context": RunnablePassthrough(), "email": RunnablePassthrough()}
        | prompt_template
        | llm
        | StrOutputParser()
    )

    return chain.invoke({"context": context, "email": input_email})

# Полный пайплайн
def email_response_pipeline(input_email, persist_directory="./chroma_db"):
    embeddings = HuggingFaceEndpointEmbeddings(model="mixedbread-ai/mxbai-embed-large-v1", task="feature-extraction")

    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )

    llm = load_qwen_model()

    context = process_email(input_email, vectordb)

    response = generate_response(input_email, context, llm)

    return response, context

if __name__ == "__main__":

    example_documents = [
        Document(
            page_content="Здравствуйте, мой заказ опаздывает на 3 дня. Что происходит?",
            metadata={"response": "Уважаемый клиент, проверяем статус вашего заказа. Свяжемся с вами в течение 2 часов с обновлением."}
        ),
        Document(
            page_content="Не пришел код активации для купленного продукта",
            metadata={"response": "Добрый день! Код активации был отправлен на вашу почту. Проверьте, пожалуйста, папку 'Спам'."}
        ),
        Document(
            page_content="Как вернуть товар?",
            metadata={"response": "Для возврата товара заполните форму на сайте в разделе 'Возвраты' и пришлите нам номер заказа."}
        )
    ]

    # Инициализация БД (раскомментировать при первом запуске)
    # init_chroma_db(example_documents)

    new_email = "Мой заказ №12345 еще не доставлен, хотя должен был быть вчера"

    response, context = email_response_pipeline(new_email)

    print("Найденные похожие письма:")
    print(context)
    print("\nСгенерированный ответ:")
    print(response)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Device set to use cpu


Найденные похожие письма:
Письмо: Здравствуйте, мой заказ опаздывает на 3 дня. Что происходит?
Ответ: Уважаемый клиент, проверяем статус вашего заказа. Свяжемся с вами в течение 2 часов с обновлением.

Письмо: Не пришел код активации для купленного продукта
Ответ: Добрый день! Код активации был отправлен на вашу почту. Проверьте, пожалуйста, папку 'Спам'.

Письмо: Как вернуть товар?
Ответ: Для возврата товара заполните форму на сайте в разделе 'Возвраты' и пришлите нам номер заказа.

Сгенерированный ответ:
Для начала, я рассмотрю письмо от пользователя, которое было отправлено ранее:

{'context': "Письмо: Здравствуйте, мой заказ опаздывает на 3 дня. Что происходит?\nОтвет: Уважаемый клиент, проверяем статус вашего заказа. Свяжемся с вами в течение 2 часов с обновлением.\n\nПисьмо: Не пришел код активации для купленного продукта\nОтвет: Добрый день! Код активации был отправлен на вашу почту. Проверьте, пожалуйста, папку 'Спам'.\n\nПисьмо: Как вернуть товар?\nОтвет: Для возврата товара з